In [ ]:
!pip install psycopg2-binary

In [2]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [5]:
user = "admin"
password = "123456"
host = "127.0.0.1"
port = "5432"
database = "postgres"

In [6]:
try:
    conexion = psycopg2.connect(user=user, password=password, host=host, port=port, database=database)
    conexion.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)  
    
    cursor = conexion.cursor()
except (Exception, psycopg2.Error) as error:
    print("Error al conectar a PostgreSQL", error)
finally:
    # if (conexion):
    #     cursor.close()
    #     conexion.close()
    print("La conexión a PostgreSQL está cerrada")
    
    

Error al conectar a PostgreSQL connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "admin"



NameError: name 'conexion' is not defined

In [ ]:


# Paso 3: Establecer la conexión
try:
    conexion = psycopg2.connect(
        user = "tu_usuario", # Reemplaza con tu usuario de PostgreSQL
        password = "tu_contraseña", # Reemplaza con tu contraseña de PostgreSQL
        host = "127.0.0.1", # Este es el host local por defecto para PostgreSQL
        port = "5432", # Este es el puerto por defecto para PostgreSQL
        database = "nombre_de_tu_base_de_datos" # Reemplaza con el nombre de tu base de datos
    )
    
    # Paso 4: Crear un cursor
    cursor = conexion.cursor()
    
    # Aquí puedes comenzar a ejecutar comandos SQL usando el cursor...
    
except (Exception, psycopg2.Error) as error:
    print("Error al conectar a PostgreSQL", error)
finally:
    if (conexion):
        cursor.close()
        conexion.close()
        print("La conexión a PostgreSQL está cerrada")